# Retail Object Detection

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

### Sample prediction of FAN-Small+ DINO model
Sample images are smudged

<img align="left" src="sample.jpg" width="400">
<img align="right" src="sample2.jpg" width="600">

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train a Retail Object Detection model on a retail dataset
* Evaluate the trained model
* Run inference with the trained model and visualize the result
* Export the trained model to a .onnx file for deployment to DeepStream
* Generate TensorRT engine using tao-deploy and verify the engine through evaluation

At the end of this notebook, you will have generated a trained `dino` model
which you may deploy via [DeepStream](https://developer.nvidia.com/deepstream-sdk).

## Table of Contents

This notebook shows an example usecase of DINO using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate a trained model](#head-5)
6. [Visualize inferences](#head-6)
7. [Deploy](#head-7)

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The following Notebook requires an env variable called the `$LOCAL_PROJECT_DIR` be set as the path to your workspace. The dataset that runs this Notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals are output to `$LOCAL_PROJECT_DIR/dino/results`.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they must to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Use it to configure your specific directories so that data, specs, results, and cache directories are correctly visible to the Docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=/path/to/local/tao-experiments

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "retail_object_detection", "results")
os.environ["HOST_MODEL_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "retail_object_detection", "models")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/dino

# The sample spec files are present in the same path as the downloaded samples.
os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR
! mkdir -p $HOST_MODEL_DIR

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tao_configs = {
   "Mounts":[
         # Mapping the Local project directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
       {
           "source": os.environ["HOST_MODEL_DIR"],
           "destination": "/model"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         },
        "user": "{}:{}".format(os.getuid(), os.getgid()),
        "network": "host"
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tao_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a Python package distributed as a Python wheel listed in the `nvidia-pyindex` Python index. Install the launcher by executing the following cell.

TAO Toolkit recommends that you run the TAO launcher in a virtual env with Python 3.6.9, following [these instructions](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a Python virtual env with the `virtualenv` and `virtualenvwrapper` packages. After setup of virtualenvwrapper, set the version of Python for use in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. By running:

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO Python package, you must meet the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

After you have installed the pre-requisites, log in to the Docker registry nvcr.io using the following the command:

```sh
docker login nvcr.io
```

Enter a username and password when prompted. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/gpu-cloud/ngc-user-guide/index.html#generating-api-key) to generate your own API key.

TAO Toolkit recommends that you run the TAO launcher in a virtual env with Python 3.6.9+, following [these instructions](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a Python virtual env with the `virtualenv` and `virtualenvwrapper` packages.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

Here we use [Retail Product Checkout dataset](https://www.kaggle.com/datasets/diyer22/retail-product-checkout-dataset) to illustrate the method of training the DINO model for retail object detection.

In [ ]:
# [Action required] Download the dataset manually.
# [Action required] Put your downloaded .zip dataset file at $HOST_DATA_DIR/retail-product-checkout-dataset.zip

In [ ]:
# Verification
!mkdir $HOST_DATA_DIR/retail_object_detection
!unzip $HOST_DATA_DIR/retail-product-checkout-dataset.zip -d $HOST_DATA_DIR/retail_object_detection

In [ ]:
# process raw data for binary detection task
!python $NOTEBOOK_ROOT/process_retail_data_labels.py

! echo ===================================
!ls $HOST_DATA_DIR/retail_object_detection

### 2.2 Download pre-trained model

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [ ]:
# [Action required] Uncomment below code if you didn't have NGC CLI yet
# # Installing NGC CLI on the local machine.
# ## Download and install
# %env CLI=ngccli_cat_linux.zip
# !mkdir -p $LOCAL_PROJECT_DIR/ngccli

# # Remove any previously existing CLI installations
# !rm -rf $LOCAL_PROJECT_DIR/ngccli/*
# !wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
# !unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
# !rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
# os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/retail_object_detection:*

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/retail_object_detection:trainable_binary_v2.1.1 --dest $HOST_MODEL_DIR/

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_MODEL_DIR

## 3. Provide training specification <a class="anchor" id="head-3"></a>

We provide specification files to configure the training parameters including:

* dataset: configure the dataset and augmentation methods
    * train_data_sources:
        * image_dir: the root directory for train images
        * json_file: annotation file for train data. Must be in COCO JSON format.
    * val_data_sources: 
        * image_dir: the root directory for validation images
        * json_file: annotation file for validation data. Must be in COCO JSON format.
    * num_classes: number of classes of your training data
    * batch_size: batch size for dataloader
    * workers: number of workers to do data loading
* model: configure the model setting
    * pretrained_backbone_path: path to the pretrained backbone model. ResNet50, FAN-variants, and GCViT-variants are supported
    * num_feature_levels: number of feature levels used from backbone
    * dec_layers: number of decoder layers
    * enc_layers: number of encoder layers
    * num_queries: number of queries for the model
    * num_select: number of top-k proposals to select from
    * use_dn: flag to enable denoising during training
    * dropout_ratio: drop out ratio
* train: configure the training hyperparameters
    * pretrained_model_path: load pretrained model path before train
    * freeze: freezes listed modules dutraining train
    * num_gpus: number of gpus 
    * num_nodes: number of nodes (num_nodes=1 for single node)
    * val_interval: validation interval
    * optim:
        * lr_backbone: learning rate for backbone
        * lr: learning rate for the rest of the model
        * lr_steps: learning rate decay step milestone (MultiStep)
    * num_epochs: number of epochs
    * activation_checkpoint: recompute activations in the backward to save GPU memory. Default is `True`.
    * precision: If set to fp16, the training is run on Automatic Mixed Precision (AMP)
    * distributed_strategy: Default is `ddp`. `ddp_sharded` is also supported.

See the [TAO documentation - DINO](https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/dino.html) to get all the parameters that are configurable.


In [ ]:
!cat $HOST_SPECS_DIR/train.yaml

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* Evaluation uses COCO metrics. For more info, please refer to: https://cocodataset.org/#detection-eval

* To speed up training, try setting `train.precision=fp16` for mixed precision training

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results
%env MODEL_DIR = /model

In [ ]:
print("For multi-GPU, change num_gpus in train.yaml based on your machine or pass --gpus to the cli.")
print("For multi-node, change num_gpus and num_nodes in train.yaml based on your machine or pass --num_nodes to the cli.")
# If you face out of memory issue, you may reduce the batch size in the spec file by passing dataset.batch_size=2
!tao model dino train \
          -e $SPECS_DIR/train.yaml \
          results_dir=$RESULTS_DIR/

In [ ]:
print('Trained checkpoints:')
print('---------------------')
!ls -ltrh $HOST_RESULTS_DIR/train

In [ ]:
# You can set NUM_EPOCH to the epoch corresponding to any saved checkpoint
%env NUM_EPOCH=000

In [ ]:
# Get the name of the checkpoint corresponding to your set epoch
tmp=!ls $HOST_RESULTS_DIR/train/*.pth | grep epoch=$NUM_EPOCH
%env CHECKPOINT={tmp[0]}

print('Rename a trained model: ')
print('---------------------')
!cp $CHECKPOINT $HOST_RESULTS_DIR/train/retail_object_detection.pth
!ls -ltrh $HOST_RESULTS_DIR/train/retail_object_detection.pth

## 5. Evaluate a trained model <a class="anchor" id="head-5"></a>

In this section, we run the `evaluate` tool to evaluate the trained model and produce the mAP metric.

We provide evaluate.yaml specification files to configure the evaluate parameters including:

* model: configure the model setting
    * this config should remain same as your trained model's configuration.
* dataset: configure the dataset and augmentation methods
    * test_data_sources:
        * image_dir: the root directory for evaluatation images    
        * json_file: annotation file for evaluation data. Must be in COCO JSON format.
    * num_classes: number of classes you used for training
    * eval_class_ids: classes you would like to evaluate. \
                    Note that current config file will evaluate only on class 1 (person in COCO dataset)\
                    If you remove this from config file, it will evaluate and compute the average over entire classes.
    * batch_size
    * workers
* evaluate:
    * num_gpus: number of gpus
    * conf_threshold: a threshold for confidence scores

* **NOTE: You need to change the model path in evaluate.yaml file based on your setting.**

In [ ]:
# Evaluate on TAO model
!tao model dino evaluate \
            -e $SPECS_DIR/evaluate.yaml \
            evaluate.checkpoint=$RESULTS_DIR/train/retail_object_detection.pth \
            results_dir=$RESULTS_DIR/

## 6. Visualize Inferences <a class="anchor" id="head-6"></a>
In this section, we run the `inference` tool to generate inferences on the trained models and visualize the results. The `inference` tool produces annotated image outputs and txt files that contain prediction information.

We provide evaluate.yaml specification files to configure the evaluate parameters including:

* model: configure the model setting
    * this config must remain the same as your trained model's configuration
* dataset: configure the dataset and augmentation methods
    * infer_data_sources:
        * image_dir: the list of directories for inference images
        * classmap: 
    * num_classes: number of classes you used for training
    * batch_size
    * workers
* inference
    * conf_threshold: the confidence score threshold
    * color_map: the color mapping for each class. The predicted bounding boxes will be drawn with mapped color for each class
* **NOTE: You need to change the model path in infer.yaml file based on your setting.**

In [ ]:
# # copy classmap to annotation directory
# !cp $HOST_SPECS_DIR/classmap.txt $HOST_DATA_DIR/raw-data/annotations/

In [ ]:
!tao model dino inference \
        -e $SPECS_DIR/infer.yaml \
        inference.checkpoint=$RESULTS_DIR/train/retail_object_detection.pth \
        results_dir=$RESULTS_DIR/

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg']

def visualize_images(output_path, num_cols=4, num_images=10):
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
IMAGE_DIR = os.path.join(os.environ['HOST_RESULTS_DIR'], "inference", "images_annotated")
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 6 # number of images to visualize.

visualize_images(IMAGE_DIR, num_cols=COLS, num_images=IMAGES)

## 7. Deploy <a class="anchor" id="head-7"></a>

In [ ]:
# Export the RGB model to ONNX model
!tao model dino export \
           -e $SPECS_DIR/export.yaml \
           export.checkpoint=$RESULTS_DIR/train/retail_object_detection.pth \
           export.onnx_file=$RESULTS_DIR/export/retail_object_detection.onnx \
           results_dir=$RESULTS_DIR/

In [ ]:
# Generate TensorRT engine using tao deploy
!tao deploy dino gen_trt_engine -e $SPECS_DIR/gen_trt_engine.yaml \
                               gen_trt_engine.onnx_file=$RESULTS_DIR/export/retail_object_detection.onnx \
                               gen_trt_engine.trt_engine=$RESULTS_DIR/gen_trt_engine/retail_object_detection.engine \
                               results_dir=$RESULTS_DIR

In [ ]:
# Evaluate with generated TensorRT engine
!tao deploy dino evaluate -e $SPECS_DIR/evaluate.yaml \
                              evaluate.trt_engine=$RESULTS_DIR/gen_trt_engine/retail_object_detection.engine \
                              results_dir=$RESULTS_DIR/

In [ ]:
# Inference with generated TensorRT engine
!tao deploy dino inference -e $SPECS_DIR/infer.yaml \
                              inference.trt_engine=$RESULTS_DIR/gen_trt_engine/retail_object_detection.engine \
                              results_dir=$RESULTS_DIR/

In [ ]:
# Visualizing the sample images.
IMAGE_DIR = os.path.join(os.environ['HOST_RESULTS_DIR'], "trt_inference", "images_annotated")
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 6 # number of images to visualize.

visualize_images(IMAGE_DIR, num_cols=COLS, num_images=IMAGES)

This notebook has come to an end.